In [ ]:
# install ezTrack following the instructions from the developer 
# https://github.com/denisecailab/ezTrack/wiki/Installation
# adapted from :
#Pennington ZT, Dong Z, Feng Y, Vetere LM, Page-Harley L, Shuman T, Cai DJ (2019). 
#ezTrack: An open-source video analysis pipeline for the investigation of animal behavior. Scientific Reports: 9(1): 19979

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import holoviews as hv
import numpy as np
import pandas as pd
import LocationTracking_Functions as lt
import cv2 # to calculate frame rate (mod)
import matplotlib.pyplot as plt

In [ ]:
video_dict = { 
    'dpath'        :  ("folder"),    
    'file'         : '8919-alley-dam-125-145.avi',
    'start'        : 2380, 
    'end'          : 4814, 
    'region_names' : None,
    'dsmpl'        : 1,
    'stretch'      : dict(width=1, height=1)
}

In [ ]:
# Construct the full path to the video file
full_path = video_dict['dpath'] + video_dict['file']
print(repr(full_path))

# Open the video file
video = cv2.VideoCapture(full_path)

# Check if the video was successfully opened
if video.isOpened():
    # Get the frame rate
    fps = video.get(cv2.CAP_PROP_FPS)
    print(f"The frame rate is: {fps} FPS")
else:
    print("Error: Could not open video.")

# Release the video capture object
video.release()

In [ ]:
#100 is the standard size. 200 will produce an image 2x the size, and so on.
%%output size = 150 

img_crp, video_dict = lt.LoadAndCrop(video_dict, cropmethod='Box')
img_crp

In [ ]:
#Define Reference Frame for Location Tracking

%%output size = 100

video_dict['altfile'] = 'EmptyBox.avi' 

video_dict['reference'], img_ref = lt.Reference(video_dict, num_frames=50, altfile=True, frames=[0]) 
img_ref

In [ ]:
# Define real-world distance between points

distance = 10 #set the width of the linear track as 10 cm
scale = 'cm'

video_dict['scale'] = lt.setScale(distance, scale, video_dict['scale'])

print(video_dict['scale'])

# Calculate pixels per cm
pixels_per_cm = video_dict['scale']['px_distance'] / video_dict['scale']['true_distance']

# Print the result
print(f"Pixels per cm: {pixels_per_cm}")

In [ ]:
# Set Location Tracking Parameters
tracking_params = {
    'loc_thresh'    : 99.5, 
    'use_window'    : True, 
    'window_size'   : 100, 
    'window_weight' : .9, 
    'method'        : 'dark',
    'rmv_wire'      : True, 
    'wire_krn'      : 5
}

In [ ]:
#Track Location and Save Results to .csv File
location = lt.TrackLocation(video_dict, tracking_params)   
location.to_csv(os.path.splitext(video_dict['fpath'])[0] + '_LocationOutput.csv', index=False)
location.head()

In [ ]:
#import z-score file
z_scores = pd.read_excel("")
specific_time_points = [125, 146.7]  # Add or remove time points as needed

In [ ]:
# Create the main figure 2G
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot Z-Score vs. Time_aligned on the main axis using adjusted_data
ax1.plot(z_scores['Time_aligned'], z_scores['Z-Score'], label='Z-Score', color='blue')
ax1.set_xlabel('Time_aligned')
ax1.set_ylabel('Z-Score')
ax1.set_title('Z-Score vs. Time')
ax1.grid(True)

# Add a thicker line at Time_aligned is zero
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Time_aligned=0')

# Create a second x-axis at the top that shares the same y-axis
ax2 = ax1.twiny()

# Set the tick positions and labels for the second x-axis to represent Time_original
ax2.set_xlim(ax1.get_xlim())
ax2.set_xlabel('Time_original')

# Add thicker lines for specific time points in Time_original
for time_point in specific_time_points:
    # Add a vertical line at this Time_aligned position
    ax1.axvline(x= time_point color='green', linestyle='--', linewidth=2, label=f'Time_original={time_point}')

# Adjust the legend to include the newly added lines
ax1.legend()

# Show the plot
plt.show()

In [ ]:
# Create the main figure 2F

location = pd.read_csv('LocationOutput.csv')

X = location['X'];
Y = location['Y'];
Z = z_scores['Z-Score']

# Slicing from index 100 to 600
X = X[99:600]  
Y = Y[99:600]
Z = Z[99:600]

# Scatter plot with colormap
plt.scatter(X, Y, s=200, c=Z, cmap='viridis', alpha=0.5)  # 's' specifies marker size
plt.colorbar()  # Add colorbar

# Set the limits of the colormap
plt.clim(0, 5)  

# Add points (gate opening, first touch)
# plt.scatter(X[100], Y[100], s=300, c="red")
# plt.scatter(X[440], Y[440], s=300, c="red")

# Set the axes limits
plt.xlim(0, 600)  # Set X-axis limits
plt.ylim(0, 200)  # Set Y-axis limits

# Ensure the axes maintain the same scale
plt.axis('equal')

plt.show()